# MISA
Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References


---

In [1]:
!pip install scikit-learn==1.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.17.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.3 which is incompatible.


In [2]:
from sklearn.decomposition  import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston, load_iris, load_breast_cancer, make_blobs
import numpy as np
from random import randrange
from sklearn.metrics import accuracy_score
import cvxpy as cp

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = load_iris()
X, y = data.data, data.target

# PCA

In [3]:
class PrincipalComponentAnalysis():
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None

    def fit(self, X):

            U,S,V = np.linalg.svd(X)
            self.components =V.T[:, :self.n_components]


    def transform(self, X):
        # YOUR CODE HERE
        z = X.dot(self.components)
        return z

In [4]:
X_centered = X - np.mean(X, axis=0)

pca = PCA(n_components=3)
pca.fit(X_centered)
X_pca_trans = pca.transform(X_centered)

model = PrincipalComponentAnalysis(n_components=3)
model.fit(X_centered)
X_model_trans = model.transform(X_centered)

sign_correct_X_model_trans = np.concatenate([np.expand_dims(X_model_trans[:,0],axis=1),-X_model_trans[:,1:]],axis=1)

error = rel_error(X_pca_trans, sign_correct_X_model_trans)
print(error)
assert  error < 1e-11

1.53330964952525e-13


# Binary Linear SVM with CVXPY

## Hard margin

In [5]:
X2, y2 = make_blobs(n_samples=300, centers=2, n_features=12, random_state=47)
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)
y2[y2 == 0] = -1

$$\min_{\mathbf{w},b}\frac{1}{2}||\mathbf{w}||^2$$ <br>
$$\text{s.t } y_i(\mathbf{w}^{\top}\mathbf{x}_i + b) \ge 1, \ i=1...N$$

In [14]:
class LinearSVMHardMargin():
    def __init__(self):
        self.w = None
        self.b = 0

    def fit(self, X, y):
        # YOUR CODE HERE
        # raise NotImplementedError()
        n = X.shape[0]
        d = X.shape[1]

        w = cp.Variable((d,1))
        b = cp.Variable((n,1))

        c =cp.norm(w) ** 2
        f = (1/2) * (c)
        a = cp.Minimize(f)
        prob = cp.Problem(a, [y @ (X @ w + b) >= 1])
        prob.solve()

        self.w = w.value
        self.b = b.value

    def predict(self, X):
        """Return the predicted label 1 or -1"""
        # YOUR CODE HERE
        # raise NotImplementedError()
        y_pred = (X.dot(self.w) + self.b)

        y_pred[y_pred > 0] = 1
        y_pred[y_pred < 0] = -1

        return y_pred

In [15]:
model = LinearSVMHardMargin()
model.fit(X2, y2)
pred = model.predict(X2)
accuracy = accuracy_score(y2, pred)
print(accuracy)
assert accuracy == 1

1.0


## Soft Margin

In [16]:
data3 = load_breast_cancer()
X3, y3 = data3.data, data3.target
scaler = StandardScaler()
X3 = scaler.fit_transform(X3)
y3[y3 == 0] = -1

$$L(\mathbf{w},b) = \frac{1}{N} \sum_{i=1}^N \max(0, y_i(\mathbf{w}^{\top}\mathbf{x}_i + b)) + \lambda||\mathbf{w}||^2_2$$

In [20]:
class LinearSVMSoftMargin():
    def __init__(self, alpha=0):
        self.w = None
        self.b = 0
        self.alpha = alpha

    def fit(self, X, y):

        # YOUR CODE HERE
        #raise NotImplementedError()
        n = len(y)
        d= X.shape[1]

        w = cp.Variable((d,1))
        b= cp.Variable((n,1))


        c = y @ (X @ w+b)
        e = cp.norm2(w)**2
        L = (1/n)*cp.sum(cp.pos(1-c))+ self.alpha*(e)

        prob = cp.Problem(cp.Minimize(L))
        prob.solve()

        self.w = w.value
        self.b = b.value
    def predict(self, X):

        """Return the predicted label 1 or -1"""
        # YOUR CODE HERE
        y_pred = (X.dot(self.w) + self.b)

        y_pred[y_pred > 0] = 1
        y_pred[y_pred < 0] = -1

        return y_pred

In [21]:
model = LinearSVMSoftMargin(alpha=1e-3)
model.fit(X3, y3)
pred = model.predict(X3)
accuracy = accuracy_score(y3, pred)
print(accuracy)
assert accuracy >= 0.98

1.0


# Multiclass Linear SVM

## Loss

$$L(\mathbf{W}) = \sum_{i=1}^N \sum_{j \neq y_i} \max(0, s_j - s_{y_i} + 1) + \lambda||\mathbf{w}||^2_2$$ <br>
$$\text{where } s_j = (f(\mathbf{x}_i;\mathbf{W}))_j = (\mathbf{W}\mathbf{x}_i)_j \text{ is the score for the j-th class}$$

In [22]:
W = np.random.randn(X.shape[1], 3) * 0.0001

In [52]:
def svm_loss_naive(W, X, y, alpha):
    """
    Multiclass SVM loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """

    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)
    num_classes = W.shape[1]
    num_train = X.shape[0]
    loss = 0.0


    for i in range(num_train):
        scores = X[i].dot(W)
        correct_class_score = scores[y[i]]

        for j in range(num_classes):
            if j == y[i]:
                continue

            margin = scores[j] - correct_class_score + 1

            if margin>0:
                loss += margin
                dW[:,y[i]] -= X[i,:]
                dW[:,j] += X[i,:]

    # Averaging over all examples
    loss /= num_train
    dW /= num_train

    # Add regularization
    s = alpha * np.sum(W * W)
    loss += 0.5 * s
    dW += alpha * W

    return loss, dW



In [53]:
# NO REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 0.0)

f = lambda W: svm_loss_naive(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -0.370667 analytic: -0.370667, relative error: 2.136525e-11
numerical: -0.744667 analytic: -0.744667, relative error: 4.991330e-11
numerical: -0.092667 analytic: -0.092667, relative error: 1.433723e-10
numerical: -0.502000 analytic: -0.502000, relative error: 2.214475e-12
numerical: 0.083333 analytic: 0.083333, relative error: 3.964115e-10
numerical: 0.083333 analytic: 0.083333, relative error: 3.964115e-10
numerical: -0.744667 analytic: -0.744667, relative error: 4.991330e-11
numerical: 0.083333 analytic: 0.083333, relative error: 3.964115e-10
numerical: 0.083333 analytic: 0.083333, relative error: 3.964115e-10
numerical: -0.092667 analytic: -0.092667, relative error: 1.433723e-10
numerical: 0.837333 analytic: 0.837333, relative error: 2.852025e-11
numerical: -1.794000 analytic: -1.794000, relative error: 9.448159e-12


In [29]:
#REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 2)

f = lambda W: svm_loss_naive(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -0.502054 analytic: -0.502054, relative error: 9.874845e-13
numerical: 0.287395 analytic: 0.287395, relative error: 1.534684e-10
numerical: -0.744710 analytic: -0.744710, relative error: 4.596918e-11
numerical: -1.794161 analytic: -1.794161, relative error: 9.807327e-12
numerical: -0.826312 analytic: -0.826312, relative error: 3.023860e-11
numerical: -1.794161 analytic: -1.794161, relative error: 9.807327e-12
numerical: 0.953044 analytic: 0.953044, relative error: 4.058021e-11
numerical: -0.126266 analytic: -0.126266, relative error: 1.634139e-11
numerical: -0.370462 analytic: -0.370462, relative error: 2.780455e-11
numerical: 0.837592 analytic: 0.837592, relative error: 2.923900e-11
numerical: 0.287395 analytic: 0.287395, relative error: 1.534684e-10
numerical: -0.092697 analytic: -0.092697, relative error: 1.292553e-10


In [30]:
def svm_loss_vectorized(W, X, y, alpha):
    """
    Multiclass SVM loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)
    # raise NotImplementedError()

    # CODE FROM
    # https://mlxai.github.io/2017/01/06/vectorized-implementation-of-svm-loss-and-gradient-update.html

    num_train = X.shape[0]

    scores = X.dot(W)
    yi_scores = scores[np.arange(scores.shape[0]),y] # http://stackoverflow.com/a/23435843/459241
    margins = np.maximum(0, scores - np.matrix(yi_scores).T + 1)
    margins[np.arange(num_train),y] = 0
    loss = np.mean(np.sum(margins, axis=1))
    loss += 0.5 * alpha * np.sum(W * W)

    binary = margins
    binary[margins > 0] = 1
    row_sum = np.sum(binary, axis=1)
    binary[np.arange(num_train), y] = -row_sum.T
    dW = np.dot(X.T, binary)

    # Average
    dW /= num_train

    # Regularize
    dW += alpha * W
    # YOUR CODE HERE


    return loss, dW

In [31]:
# NO REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 0.0)

f = lambda W: svm_loss_vectorized(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: 2.296000 analytic: 2.296000, relative error: 4.480543e-12
numerical: -0.370667 analytic: -0.370667, relative error: 8.587256e-12
numerical: -0.092667 analytic: -0.092667, relative error: 9.625027e-11
numerical: -0.826667 analytic: -0.826667, relative error: 1.290500e-12
numerical: -0.092667 analytic: -0.092667, relative error: 9.625027e-11
numerical: -0.826667 analytic: -0.826667, relative error: 1.290500e-12
numerical: 0.953333 analytic: 0.953333, relative error: 1.149896e-12
numerical: 0.083333 analytic: 0.083333, relative error: 1.299530e-10
numerical: 0.953333 analytic: 0.953333, relative error: 1.149896e-12
numerical: -0.370667 analytic: -0.370667, relative error: 8.587256e-12
numerical: 0.083333 analytic: 0.083333, relative error: 1.299530e-10
numerical: -1.794000 analytic: -1.794000, relative error: 1.651101e-13


In [32]:
# REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 2)

f = lambda W: svm_loss_vectorized(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: 0.287395 analytic: 0.287395, relative error: 2.037028e-11
numerical: 0.837592 analytic: 0.837592, relative error: 3.898145e-12
numerical: -0.092697 analytic: -0.092697, relative error: 1.102878e-10
numerical: -0.092697 analytic: -0.092697, relative error: 1.102878e-10
numerical: -0.502054 analytic: -0.502054, relative error: 9.885902e-13
numerical: -0.126266 analytic: -0.126266, relative error: 1.042671e-10
numerical: -0.826312 analytic: -0.826312, relative error: 3.350909e-12
numerical: -0.744710 analytic: -0.744710, relative error: 6.210871e-12
numerical: -0.502054 analytic: -0.502054, relative error: 9.885902e-13
numerical: -1.794161 analytic: -1.794161, relative error: 5.251118e-13
numerical: -0.744710 analytic: -0.744710, relative error: 6.210871e-12
numerical: 0.287395 analytic: 0.287395, relative error: 2.037028e-11


## Gradient descent

In [34]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            # YOUR CODE HERE
            tmp = np.ones((X.shape[0], 1))
            X = np.concatenate((tmp, X), axis=1)

            #raise NotImplementedError()

        N, d = X.shape


        C = (np.max(y) + 1)
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W

        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            # raise NotImplementedError()
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice,:]
            y_batch = y[choice]

            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update
            # Update the weights w using the gradient and the learning rate.
            # YOUR CODE HERE
            # raise NotImplementedError()
            self.W -= learning_rate * dW

            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class LinearSVM(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return svm_loss_vectorized(self.W, X_batch, y_batch, alpha)

    def predict(self, X):
        """
        Inputs:
        - X: array of shape (N, D)

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class
        """
        # YOUR CODE HERE
        # raise NotImplementedError()
        if self.fit_intercept:
            tmp = np.ones((len(X),1))
            X = np.append(tmp, X, axis=1)

        z = X.dot(self.W)
        y_pred = np.argmax(z, axis=1)

        return y_pred

In [35]:
model = LinearSVM(fit_intercept=True)
model.train(X, y, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X)
model_accuracy = accuracy_score(y, pred)
print(model_accuracy)
assert model_accuracy > 0.97

iteration 0 / 75000: loss 1.999119
iteration 10000 / 75000: loss 0.143852
iteration 20000 / 75000: loss 0.143602
iteration 30000 / 75000: loss 0.100846
iteration 40000 / 75000: loss 0.115263
iteration 50000 / 75000: loss 0.114839
iteration 60000 / 75000: loss 0.049228
iteration 70000 / 75000: loss 0.034964
0.9733333333333334
